# Prepare atmospheric and ocean data

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:30:00'
cores = 4
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52+gdata/xv83'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.71.12:8787/status,
Dashboard: http://10.6.71.12:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.71.12:33815,Workers: 0
Dashboard: http://10.6.71.12:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [30]:
import xarray as xr
import numpy as np
import pandas as pd

In [6]:
import functions as fn

In [42]:
def anomalise_and_process(
    load,
    dataset,
    filepath,
    data_root_path,
    var_name,
    years,
    new_var_name,
    long_name,
    units,
    chunks
):
    if load:
        return xr.open_zarr(filepath, consolidated=True)
    else:
        if dataset == 'era5':
            ds = fn.open_era_data(
                root_path=data_root_path,
                variable=var_name,
                years=years
            )
        elif dataset == 'hadisst':
            ds = xr.open_zarr(data_root_path, consolidated=True)
            ds = ds.sel(time=slice(str(years[0]), str(years[-1])))
            ds['time'] = pd.date_range(str(years[0]), str(years[-1])+'-12-01', freq='1MS')
            ds = xr.where(ds > -2, ds, np.nan)
        else:
            raise ValueError("Incorrect dataset.")
            
        ds = ds.rename({
            'latitude': 'lat',
            'longitude': 'lon',
            var_name: new_var_name
        })
        da = ds[new_var_name]
        da = da.chunk(chunks)
            
        da_anoms = da.groupby('time.month').apply(lambda x: x - x.mean('time'))
            
        da_anoms = da_anoms.assign_attrs({
            'long_name': long_name,
            'short_name': new_var_name,
            'units': units
        })
            
        ds_anoms = da_anoms.to_dataset(name=new_var_name + '_anom')
        encoding = {new_var_name + '_anom': {'dtype': 'float32'}}
        ds_anoms.to_zarr(
            filepath,
            mode='w',
            consolidated=True,
            encoding=encoding
        )
        
        return ds_anoms

In [8]:
data_fp = '/g/data/w42/dr6273/work/data/'
years = range(1959, 2021)

# SST data

In [9]:
dataset = 'hadisst'
sst_fp = data_fp + dataset + '/sst/sst_anom_'+dataset+'_moda_sfc_'+str(years[0])+'-'+str(years[-1])+'.zarr'
print(sst_fp)

/g/data/w42/dr6273/work/data/hadisst/sst/sst_anom_hadisst_moda_sfc_1959-2020.zarr


In [46]:
sst_anoms = anomalise_and_process(
    load=True,
    dataset='hadisst',
    filepath=sst_fp,
    data_root_path='/g/data/xv83/reanalyses/HadISST/ocean_month.zarr',
    var_name='sst',
    years=years,
    new_var_name='sst',
    long_name='Sea surface temperature',
    units='K',
    chunks={'time': 12, 'lat': -1, 'lon': -1}
)

### Nino 3.4

In [47]:
nino34 = sst_anoms.sst_anom.sel(
    lat=slice(5, -5),
    lon=slice(-170, -120)
).mean(['lat','lon']).to_dataset(name='nino34')

In [48]:
nino34_dt = fn.detrend_dim(nino34.nino34, 'time').to_dataset(name='nino34_detrended')

In [49]:
nino34 = nino34.merge(nino34_dt)

In [50]:
nino34_fp = data_fp + dataset + '/climate_modes/'+dataset+'_nino34_'+str(years[0])+'-'+str(years[-1])+'.zarr'
nino34.to_zarr(nino34_fp, mode='w', consolidated=True)

### Dipole mode index

In [51]:
def calc_dmi(da):
    """
    Calculate Dipole Mode Index
    """    
    da_W = da.sel(lat=slice(10, -10), lon=slice(50, 70)).mean(['lat', 'lon'])
    da_E = da.sel(lat=slice(0, -10), lon=slice(90, 110)).mean(['lat', 'lon'])
    
    return (da_W - da_E)

In [52]:
dmi = calc_dmi(sst_anoms.sst_anom).to_dataset(name='dmi')

In [53]:
dmi_dt = fn.detrend_dim(dmi.dmi, 'time').to_dataset(name='dmi_detrended')

In [54]:
dmi = dmi.merge(dmi_dt)

In [55]:
dmi_fp = data_fp + dataset + '/climate_modes/'+dataset+'_dmi_'+str(years[0])+'-'+str(years[-1])+'.zarr'

In [56]:
dmi.to_zarr(dmi_fp, mode='w', consolidated=True)

# MSLP data

In [8]:
dataset = 'era5'
mslp_fp = data_fp + dataset + '/mslp/mslp_anom_'+dataset+'_moda_sfc_'+str(years[0])+'-'+str(years[-1])+'.zarr'
print(mslp_fp)

/g/data/w42/dr6273/work/data/era5/mslp/mslp_anom_era5_moda_sfc_1959-2020.zarr


In [94]:
mslp_anoms = anomalise_and_process(
    load=False,
    dataset='era5',
    filepath=mslp_fp,
    data_root_path='/g/data/rt52/era5/single-levels/monthly-averaged/',
    var_name='msl',
    years=years,
    new_var_name='mslp',
    long_name='Mean sea level pressure',
    units='Pa',
    chunks={'time': 12, 'lat': -1, 'lon': -1}
)

### Southern Annular Mode index

In [26]:
def calc_sam(mslp_anoms, lat_name='lat', lon_name='lon', time_name='time'):
    mslp_40 = mslp_anoms.interp({lat_name: -40}).mean(lon_name)
    mslp_65 = mslp_anoms.interp({lat_name: -65}).mean(lon_name)
    
    norm_40 = mslp_40.groupby(time_name+'.month').apply(lambda x: x / x.std(time_name))
    norm_65 = mslp_65.groupby(time_name+'.month').apply(lambda x: x / x.std(time_name))
    
    return norm_40 - norm_65

In [27]:
sam = calc_sam(mslp_anoms)

In [31]:
sam = sam.rename({'mslp_anom': 'sam'})

In [32]:
sam_dt = fn.detrend_dim(sam.sam, 'time').to_dataset(name='sam_detrended')

In [33]:
sam = sam.merge(sam_dt)

In [36]:
sam_fp = data_fp + dataset + '/climate_modes/'+dataset+'_sam_'+str(years[0])+'-'+str(years[-1])+'.zarr'
sam.to_zarr(sam_fp, mode='w', consolidated=True)

# Close cluster

In [59]:
client.close()
cluster.close()